In [1]:
import pandas as pd
from transformers import pipeline, GPT2LMHeadModel, GPT2Tokenizer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


d:\learning\LIVE-TWITTER-SENTIMENT-ANALYSIS\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df1 = pd.read_csv('twitter_dataset.csv') 
df1["Text"] = df1["Text"].astype(str)
df1.head()

,Tweet_ID,Username,Text,Retweets,Likes,Timestamp
0,1,julie81,Party least receive say or single. Prevent pre...,2,25,2023-01-30 11:00:51
1,2,richardhester,Hotel still Congress may member staff. Media d...,35,29,2023-01-02 22:45:58
2,3,williamsjoseph,Nice be her debate industry that year. Film wh...,51,25,2023-01-18 11:25:19
3,4,danielsmary,Laugh explain situation career occur serious. ...,37,18,2023-04-10 22:06:29
4,5,carlwarren,Involve sense former often approach government...,27,80,2023-01-24 07:12:21


In [3]:
df1['Timestamp'] = pd.to_datetime(df1['Timestamp'], format="%Y-%m-%d %H:%M:%S")
df1['Timestamp'] = df1['Timestamp'].dt.strftime("%d-%m-%Y %H:%M:%S")
df1 = df1.drop(columns=["Username", "Retweets", "Tweet_ID"])
df1.rename(columns={'Text': 'text', 'Timestamp':'date'}, inplace=True)

In [4]:
df1.head()

,text,Likes,date
0,Party least receive say or single. Prevent pre...,25,30-01-2023 11:00:51
1,Hotel still Congress may member staff. Media d...,29,02-01-2023 22:45:58
2,Nice be her debate industry that year. Film wh...,25,18-01-2023 11:25:19
3,Laugh explain situation career occur serious. ...,18,10-04-2023 22:06:29
4,Involve sense former often approach government...,80,24-01-2023 07:12:21


In [5]:
df2 = pd.read_csv('elon_musk_tweets.csv')
df2 = df2.drop(columns=["id","user_name","user_description","user_created","user_followers","user_friends","user_favourites","user_verified","source", "retweets", "favorites","is_retweet"])
df2

,user_location,date,text,hashtags
0,NaN,2022-07-05 17:55:09+00:00,@BillyM2k I find the gold toe sock – inevitabl...,NaN
1,NaN,2022-07-05 17:47:42+00:00,"Sock Con, the conference for socks",NaN
2,NaN,2022-07-05 17:46:15+00:00,Always something new for the magazine cover an...,NaN
3,NaN,2022-07-05 17:40:05+00:00,@ExplainThisBob This guy gets it,NaN
4,NaN,2022-07-05 17:38:23+00:00,Sock tech is so advanced that you can get pret...,NaN
...,...,...,...,...
5899,NaN,2023-06-03 23:48:42+00:00,@JonErlichman He’s not wrong …,NaN
5900,NaN,2023-06-03 23:31:23+00:00,"@alifarhat79 Guys, I think I maybe took too mu...",NaN
5901,NaN,2023-06-03 23:23:41+00:00,@sriramk Cool,NaN
5902,NaN,2023-06-03 22:59:31+00:00,@cb_doge Time to complete the circle,NaN


In [6]:
df3 = pd.read_csv('covid19_tweets.csv')
df3 = df3.drop(columns=["user_name","user_description","user_created","user_followers","user_friends","user_favourites","user_verified","source", "is_retweet"])
df3.head()

,user_location,date,text,hashtags
0,astroworld,2020-07-25 12:27:21,If I smelled the scent of hand sanitizers toda...,NaN
1,"New York, NY",2020-07-25 12:27:17,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,NaN
2,"Pewee Valley, KY",2020-07-25 12:27:14,@diane3443 @wdunlap @realDonaldTrump Trump nev...,['COVID19']
3,Stuck in the Middle,2020-07-25 12:27:10,@brookbanktv The one gift #COVID19 has give me...,['COVID19']
4,Jammu and Kashmir,2020-07-25 12:27:08,25 July : Media Bulletin on Novel #CoronaVirus...,"['CoronaVirusUpdates', 'COVID19']"


In [7]:
import re

# data cleaning and prep
def data_cleaning_and_preparation(df, text_column):
    # 1. Drop rows with missing values in text columns
    df.dropna(subset=[text_column], inplace=True)

    # 2. Remove duplicates
    df.drop_duplicates(subset=[text_column], inplace=True)

    # 3. Convert text to lowercase
    df[text_column] = df[text_column].str.lower()

    # 4. Remove special characters, punctuation, and numimbers
    df[text_column] = df[text_column].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x))

    # 5. Remove extra whitespace
    df[text_column] = df[text_column].apply(lambda x: re.sub(r'\s+', ' ', x).strip())

    # 7. Check the cleaned data
    print(df.head())
    return df
    # Save the cleaned text data
    # df.to_csv('cleaned_text_data.csv', index=False)


In [ ]:
df1 = data_cleaning_and_preparation(df1, text_column='text')
df2 = data_cleaning_and_preparation(df2, text_column='text')
df3 = data_cleaning_and_preparation(df3, text_column='text')

In [9]:
print(type(df1))

<class 'pandas.core.frame.DataFrame'>


In [10]:
overall_df = pd.concat([df1,df2, df3], axis=1, join="outer")

In [ ]:
overall_df.head()

In [ ]:
import pandas as pd

dfs = [df1, df2, df3]
non_empty_dfs = [df for df in dfs if df is not None and not df.empty]

# Concatenate the remaining valid DataFrames
if non_empty_dfs:
    result = pd.concat(non_empty_dfs, axis=1)
    print(result)
else:
    print("No valid DataFrames to concatenate.")


In [13]:
len(result)
result.to_csv('final_df.csv')

In [24]:
df1 = df1.iloc[: cutting_size]

In [27]:
def sentiment_analysis_transformers(df):
    
    model_path = "cardiffnlp/twitter-xlm-roberta-base-sentiment-multilingual"
    analyser = pipeline("sentiment-analysis", model=model_path)
    df['scores'] = df['Text'].apply(lambda text: analyser (text.lower()))
    df['Sentiment'] = df['scores'].apply(lambda output: output[0]['label']) 
    df['Sentiment'] = df['Sentiment'].apply(lambda x: 1 if x == 'positive' else (-1 if x == 'negative' else 0)) 
    
    # df['Sentiment'] = df['Sentiment'].apply(lambda x: 'Positive' if x == 'positive' else ('Negative' if x == 'negative' else 'Neutral')) 
    
    # df['Sentiment'] = df['Sentiment'].apply(lambda x: 'Positive ❤️' if x == 'positive' else ('Negative 😡' if x == 'negative' else 'Neutral 💛')) 
    
    df =  df.drop(columns = ["scores"])
    return df

In [17]:
len(df1)

1000

In [ ]:
transformers_result = sentiment_analysis_transformers(df1)

In [21]:
transformers_result

,Tweet_ID,Text,Timestamp,Sentiment
0,1,Party least receive say or single. Prevent pre...,30-01-2023 11:00:51,Positive
1,2,Hotel still Congress may member staff. Media d...,02-01-2023 22:45:58,Neutral
2,3,Nice be her debate industry that year. Film wh...,18-01-2023 11:25:19,Positive
3,4,Laugh explain situation career occur serious. ...,10-04-2023 22:06:29,Neutral
4,5,Involve sense former often approach government...,24-01-2023 07:12:21,Neutral
...,...,...,...,...
995,996,Make environment actually feeling hit need. Be...,29-01-2023 05:36:17,Neutral
996,997,Just him investment state her either trial. Fo...,20-01-2023 13:10:51,Neutral
997,998,Similar also finish analysis.\nIndividual bene...,12-04-2023 09:51:11,Neutral
998,999,Possible resource early itself head federal. T...,28-03-2023 03:02:53,Neutral


In [39]:
df2 = pd.read_csv('Twitter_Data.csv') 
df2["clean_text"] = df2["clean_text"].astype(str)
df2

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0
...,...,...
162975,why these 456 crores paid neerav modi not reco...,-1.0
162976,dear rss terrorist payal gawar what about modi...,-1.0
162977,did you cover her interaction forum where she ...,0.0
162978,there big project came into india modi dream p...,0.0


In [40]:
test_data = df2.iloc[:cutting_size]

In [41]:
len(test_data)

1000

In [42]:
def sentiment_analysis1(df):
    
    model_path = "cardiffnlp/twitter-xlm-roberta-base-sentiment-multilingual"
    analyser = pipeline("sentiment-analysis", model=model_path)
    df['scores'] = df['clean_text'].apply(lambda text: analyser (text.lower()))
    df['Sentiment'] = df['scores'].apply(lambda output: output[0]['label']) 
    df['Sentiment'] = df['Sentiment'].apply(lambda x: 1 if x == 'positive' else (-1 if x == 'negative' else 0)) 
    
    df =  df.drop(columns = ["scores"])
    return df

In [ ]:
test_result = sentiment_analysis1(test_data)

In [49]:
from sklearn.metrics import confusion_matrix,classification_report

# Calculate the confusion matrix
conf_matrix = confusion_matrix(test_result["category"], test_result["Sentiment"])

# Calculate the accuracy
accuracy = classification_report(test_result["category"], test_result["Sentiment"])

print("Confusion Matrix:")
print(conf_matrix)
print("\nAccuracy:", accuracy)

Confusion Matrix:
[[170  38  17]
 [125 130  79]
 [182  93 166]]

Accuracy:               precision    recall  f1-score   support

        -1.0       0.36      0.76      0.48       225
         0.0       0.50      0.39      0.44       334
         1.0       0.63      0.38      0.47       441

    accuracy                           0.47      1000
   macro avg       0.50      0.51      0.46      1000
weighted avg       0.53      0.47      0.46      1000



In [51]:
# Load your CSV file
data = test_data

# Initialize VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Function to analyze sentiment and classify based on compound score
def classify_sentiment(text):
    # Get sentiment scores
    scores = analyzer.polarity_scores(text)
    
    # Get individual scores
    neg_score = scores['neg']
    neu_score = scores['neu']
    pos_score = scores['pos']
    compound_score = scores['compound']
    
    # Check if compound score is the highest
    if compound_score >= max(neg_score, neu_score, pos_score):
        # Classify based on compound score
        if compound_score >= 0.05:
            return 1
        elif compound_score > -0.05:
            return 0
        else:
            return -1
    else:
        # Classify based on the highest individual score
        highest_score = max(neg_score, neu_score, pos_score)
        if highest_score == neg_score:
            return -1
        elif highest_score == neu_score:
            return 0
        else:
            return 1
text = "This movie was okay, not great but not terrible"
print(classify_sentiment(text))

# # Apply the sentiment analysis function to the 'text' column
data['Sentiment'] = data['clean_text'].apply(classify_sentiment)
data = data.drop(columns=["scores"])
# data['Sentiment'] = data['Text'].apply(classify_sentiment)
data
# Save the results to a new CSV file
# data.to_csv('data_with_sentiment.csv', index=False)




0


C:\Users\beaut\AppData\Local\Temp\ipykernel_24748\767759645.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Sentiment'] = data['clean_text'].apply(classify_sentiment)


,clean_text,category,Sentiment
0,when modi promised “minimum government maximum...,-1.0,0
1,talk all the nonsense and continue all the dra...,0.0,0
2,what did just say vote for modi welcome bjp t...,1.0,0
3,asking his supporters prefix chowkidar their n...,1.0,0
4,answer who among these the most powerful world...,1.0,0
...,...,...,...
995,another meltdown india which will further rejo...,1.0,0
996,there are two reasons for atmosphere hatred cr...,0.0,0
997,modi has wiped out the small micro industries ...,-1.0,0
998,bjp struggles find candidates west bengal graf...,-1.0,0


In [52]:
from sklearn.metrics import confusion_matrix,classification_report

# Calculate the confusion matrix
conf_matrix = confusion_matrix(data["category"], data["Sentiment"])

# Calculate the accuracy
accuracy = classification_report(data["category"], data["Sentiment"])

print("Confusion Matrix:")
print(conf_matrix)
print("\nAccuracy:", accuracy)

Confusion Matrix:
[[  9 201  15]
 [  2 309  23]
 [  1 341  99]]

Accuracy:               precision    recall  f1-score   support

        -1.0       0.75      0.04      0.08       225
         0.0       0.36      0.93      0.52       334
         1.0       0.72      0.22      0.34       441

    accuracy                           0.42      1000
   macro avg       0.61      0.40      0.31      1000
weighted avg       0.61      0.42      0.34      1000



In [ ]:
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Load CSV file
# data = pd.read_csv('data.csv')
data_gpt  = test_data.loc[:100]

# Function to analyze sentiment
def analyze_sentiment(text):
    prompt = f"What is the sentiment of the following text? '{text}'"
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(inputs.input_ids, max_length=100, do_sample=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Apply sentiment analysis to each row in the 'text' column
data_gpt['Sentiment'] = data['clean_text'].apply(analyze_sentiment)

# # Save the results to a new CSV file
# data.to_csv('data_with_sentiment.csv', index=False)

# print("Sentiment analysis complete! Results saved to 'data_with_sentiment.csv'.")
# print(analyze_sentiment(text))
data_gpt


In [ ]:
from sklearn.metrics import confusion_matrix,classification_report

# Calculate the confusion matrix
conf_matrix = confusion_matrix(data_gpt["category"], data_gpt["Sentiment"])

# Calculate the accuracy
accuracy = classification_report(data_gpt["category"], data_gpt["Sentiment"])

print("Confusion Matrix:")
print(conf_matrix)
print("\nAccuracy:", accuracy)

In [ ]:
# hashtag gen
# topic classififcation
# data cleaning and enreichement
# define the terms of the projet ie
# sentiment anal, ETL, pipeline, real time, batch,diragrams, usecase, class, package,sequence

In [20]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import nltk
nltk.download('punkt')

tokenizer = AutoTokenizer.from_pretrained("fabiochiu/t5-base-tag-generation")
model = AutoModelForSeq2SeqLM.from_pretrained("fabiochiu/t5-base-tag-generation")
def hastag_generator(text):

    inputs = tokenizer([text], max_length=512, truncation=True, return_tensors="pt")
    output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=10,
                            max_length=64)
    decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
    tags = list(set(decoded_output.strip().split(", ")))

    return tags



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\beaut\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['Shopping', 'Food', 'Foodies']
